# Single run testing 

At initialization, asset and liabilites are equal 
$$\alpha D + \beta D + L  = \frac{1}{1-\gamma} D$$
$$\alpha + \beta \leq \frac{1}{1-\gamma}$$

## central

In [1]:
gamma=0.03
2.65*gamma

0.0795

In [ ]:
from dynamics import ClassDynamics
from network import ClassNetwork

# parameter setting
nb_banks=50
path_results="./results/single_run/test/"


# reg ratios
alpha_init=False # initial cash (< 1/(1-gamma) - beta)
alpha=0.01
beta_init=0.4 # initial collateral  (< 1/(1-gamma) - alpha)
beta_reg=0.4
beta_star=0.4
gamma=0.03
gamma_init = 2.5*gamma #3*gamma
collateral_value=1.0

# initialisation of deposits size or through money creation
initialization_method="pareto"
alpha_pareto=1.4
initial_deposits_size = 40 #40 if False, use the init money min and money creation
init_money_min = 1e-2 # 10 million money units, minimum for a bank license

# shocks on deposits
shocks_method="non-conservative"
shocks_law="normal-mean-reverting"
shocks_vol=0.05 # 0.08


# speed of learning
learning_speed = 0.5 #0.5

# min trans size
min_repo_trans_size=1e-8  # 1e-8

# dynamics & ploting
nb_steps=int(20e3) #int(10e3)
dump_period=int(5e2) #int(5e2)
plot_period=int(5e2) #int(5e2)
cp_option=True
heavy_plot=False

# LCR mgt
LCR_mgt_opt=True

# leverage mgt
end_repo_period=False # if int, periodic end repo / if false, leverage mgt
gamma_star = 1.5*gamma #1.5
check_leverage_opt = False # to avoid killing the run if one or several banks are below min leverage due to high shocks (there is not possibility of decrease balance sheet size if no interbank borrowings)

# money creation
loan_tenor=nb_steps #nb_steps # if int, money creation / if false. no new loans
loan_period=1
new_loans_vol = 10 #5 standard deviation around the mean creation of loans (if initial deposits size not False)
new_loans_mean = 2e-2/250 #2e-2/250 daily mean increase in loans expressed as a percentage of the initial loans, meaning linear growth (or of the current loans, then meaning exponential growth)
beta_new = beta_reg # if number, new colat / if false, no new colat 
gamma_new = 2.5*gamma # 2*gamma_star


# substitution of collateral
substitution = False

# Quantitative easing scenario
QE_start = False
QE_stop = False

# GFC scenario
no_trust_start = False
no_trust_stop = False

# initialize ClassNetwork
Network = ClassNetwork(
    nb_banks=nb_banks,
    initial_deposits_size=initial_deposits_size,
    alpha_init=alpha_init,
    alpha=alpha,
    beta_init=beta_init,
    beta_reg=beta_reg,
    beta_star=beta_star,
    beta_new=beta_new,
    gamma_init=gamma_init,
    gamma=gamma,
    gamma_star=gamma_star,
    gamma_new=gamma_new,
    collateral_value=collateral_value,
    initialization_method=initialization_method,
    alpha_pareto=alpha_pareto,
    shocks_method=shocks_method,
    shocks_law=shocks_law,
    shocks_vol=shocks_vol,
    LCR_mgt_opt=LCR_mgt_opt,
    min_repo_trans_size=min_repo_trans_size,
    loan_tenor=loan_tenor,
    loan_period=loan_period,
    new_loans_vol=new_loans_vol,
    new_loans_mean=new_loans_mean,
    end_repo_period=end_repo_period,
    substitution=substitution,
    learning_speed=learning_speed,
    check_leverage_opt=check_leverage_opt,
    init_money_min=init_money_min,
    QE_start=QE_start,
    QE_stop=QE_stop,
    no_trust_start=no_trust_start,
    no_trust_stop=no_trust_stop,
)

# initialize ClassDynamics
Dynamics = ClassDynamics(
    Network,
    nb_steps=nb_steps,
    path_results=path_results,
    dump_period=dump_period,
    plot_period=plot_period,
    cp_option=cp_option,
    heavy_plot=heavy_plot,
)

# simulate
Dynamics.simulate()

simulate the repo market


  2%|████▌                                                                                                                                                                                | 498/19999 [00:42<24:29, 13.27it/s]

get arr_rev_repo_exp_adj


11639it [00:02, 5631.99it/s]
  5%|█████████                                                                                                                                                                            | 998/19999 [01:46<30:43, 10.31it/s]

get arr_rev_repo_exp_adj


...

156844it [00:30, 5184.75it/s]
 42%|████████████████████████████████████████████████████████████████████████████▍                                                                                                       | 8499/19999 [51:49<57:15,  3.35it/s]

get arr_rev_repo_exp_adj


166251it [00:32, 5157.44it/s]
 45%|████████████████████████████████████████████████████████████████████████████████▉                                                                                                   | 8999/19999 [58:05<44:30,  4.12it/s]

get arr_rev_repo_exp_adj


175587it [00:33, 5165.76it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1425.87it/s]


# ploting

In [ ]:
# get graphics after interupting 
import graphics  as gx
Dynamics.nb_steps = Network.step
Dynamics.plot_period = Network.step
Dynamics.fill()
Dynamics.dump()
gx.plot(Dynamics)